In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import json, time, os, sys, glob
import shutil
import warnings
import numpy as np
import pandas as pd
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset
import copy
import torch.nn as nn
import torch.nn.functional as F
import random
import os.path
import subprocess
from tqdm import tqdm
from omegaconf import OmegaConf
import pytorch_lightning as pl
from terrace.batch import DataLoader
from typing import Sequence

from fireprot_dataset import *
from training.model_utils import featurize, loss_smoothed, loss_nll, get_std_opt, ProteinMPNN
from training.utils import worker_init_fn, get_pdbs, loader_pdb, build_training_clusters, PDB_dataset, StructureDataset, StructureLoader
from protein_mpnn_utils import loss_nll, loss_smoothed, gather_edges, gather_nodes, gather_nodes_t, cat_neighbors_nodes, _scores, _S_to_seq, tied_featurize, parse_PDB
from protein_mpnn_utils import StructureDataset, StructureDatasetPDB, ProteinMPNN
from kaggle_dataset import KaggleTrainDataset
from transfer_model import *
from train import *
from combo_dataset import *

In [13]:
cfg = OmegaConf.load("config.yaml")

In [14]:
from mega_scale_dataset import MegaScaleDataset
dataset = MegaScaleDataset(cfg, "val")

/home/boris/miniconda3/envs/chem/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3377: DtypeWarning: Columns (30,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


1GYZ.pdb
1MHN.pdb
2D1U.pdb
7JJK.pdb
1CSQ.pdb_F49A
2LSS.pdb_G6S
2LSS.pdb_F17A
1MJC.pdb_L43S
1MJC.pdb_G17A
1CSQ.pdb_F9S
2LXK.pdb_G4A
2LXK.pdb_F15S
2K5N.pdb
2MA4.pdb
1JIC.pdb
2YSC.pdb
v2_2LXE.pdb
2JVD.pdb
5UOI.pdb
2M7O.pdb_I21A
2YSB.pdb
2LT1.pdb
1YG0.pdb
6YSE.pdb
1IFY.pdb
2MC5.pdb
1O6X.pdb
2LQK.pdb
2N88.pdb
5VNT.pdb
3MYC.pdb
2RJV.pdb
1YU5.pdb
v2R14S|R16S_2L3X.pdb
2KFV.pdb
2HBB.pdb_pross4
2HBB.pdb_pross9
2HBB.pdb_pross6
2HBB_con
2HBB.pdb_pross7
2HBB.pdb_K12M
2HBB.pdb
5KPH.pdb
3L1X.pdb
2JRO.pdb
2LJ3.pdb
4HCK.pdb
2KGT.pdb
6SCW.pdb
1Y0M.pdb
2KR3.pdb
2J6K_con
6NMW.pdb
1QLY.pdb


In [17]:
len(dataset)